In [ ]:
#r "nuget: Parquet.Net, 5.0.2"

using Parquet;
using Parquet.Data;
using Parquet.Schema;
using System.Net.Http;
using System.Threading.Tasks;
using System.IO;
using System.Collections.Generic;
using System.Linq;
using System.Reflection;

string taxiUrl = "https://blobs.duckdb.org/data/taxi-data-2019.parquet";
string zoneUrl = "https://blobs.duckdb.org/data/zone-lookups.parquet";
var taxiFile = "taxi-data-2019.parquet";
var zoneFile = "zone-lookups.parquet";


Installed Packages Parquet.Net, 5.0.2

In [ ]:
async Task SaveFileToDisk(string url, string fileName)
{
    using HttpClient client = new HttpClient();
    using var stream = await client.GetStreamAsync(url);

        // Save the stream to a file on disk
        using var fileStream = new FileStream(fileName, FileMode.Create, FileAccess.Write, FileShare.None);
        await stream.CopyToAsync(fileStream);

}




In [ ]:
var sw = System.Diagnostics.Stopwatch.StartNew();
await SaveFileToDisk(taxiUrl, taxiFile);
$"Downloaded {taxiFile} in {sw.ElapsedMilliseconds}ms"

Downloaded taxi-data-2019.parquet in 29334ms

In [ ]:
var sw = System.Diagnostics.Stopwatch.StartNew();
await SaveFileToDisk(zoneUrl, zoneFile);
$"Downloaded {zoneFile} in {sw.ElapsedMilliseconds}ms"

Downloaded zone-lookups.parquet in 350ms

In [ ]:


public async Task<IReadOnlyCollection<T>> LoadParquetDataAsync<T>(string fileName) where T : new()
{
    var properties = typeof(T).GetProperties().ToDictionary(x => x.Name, x => x.SetMethod);
    using var fileStream = new FileStream(fileName, FileMode.Open, FileAccess.Read, FileShare.None);

    var options = new ParquetOptions { TreatByteArrayAsString = true };
    using var parquetReader = await ParquetReader.CreateAsync(fileStream, options);

    var data = new List<T>();

    for (int i = 0; i < parquetReader.RowGroupCount; i++)
    {
        using (ParquetRowGroupReader groupReader = parquetReader.OpenRowGroupReader(i))
        {
            var dataFields = parquetReader.Schema.GetDataFields();
            var rowCount = (int)groupReader.RowCount;
            var instances = Enumerable.Range(0, rowCount)
            .Select(_ => new T())
            .ToArray();
            data.AddRange(instances);        

            foreach (var field in dataFields)
            {
                if (properties.TryGetValue(field.Name, out var setter))
                {
                    var column = await groupReader.ReadColumnAsync(field);
                    for(long rowIndex = 0; rowIndex < rowCount; rowIndex++)
                    {
                        var value = column.Data.GetValue(rowIndex);
                        setter.Invoke(instances[rowIndex], new[] { value });
                    }

                }
            }


        }
    }

    return data;
}

In [ ]:
record TaxiTrip
{
    public string PickupBorough { get; set; }
    public string DropoffBorough { get; set; }
    public long pickup_location_id { get; set; }
    public long dropoff_location_id { get; set; }
}

record Zone
{
    public long LocationID { get; set; }
    public string Borough { get; set; }
}

In [ ]:



var sw = System.Diagnostics.Stopwatch.StartNew();
var trips = await LoadParquetDataAsync<TaxiTrip>(taxiFile);

$"Time to load file and instantiate: {sw.ToString()}"


Time to load file and instantiate: 00:01:41.5484720

In [ ]:
trips.Take(10)

index value 0 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 164, dropoff_location_id = 233 } PickupBorough <null> DropoffBorough <null> pickup_location_id 164 dropoff_location_id 233 1 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 234, dropoff_location_id = 231 } PickupBorough <null> DropoffBorough <null> pickup_location_id 234 dropoff_location_id 231 2 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 229, dropoff_location_id = 162 } PickupBorough <null> DropoffBorough <null> pickup_location_id 229 dropoff_location_id 162 3 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 234, dropoff_location_id = 249 } PickupBorough <null> DropoffBorough <null> pickup_location_id 234 dropoff_location_id 249 4 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 249, dropoff_location_id = 4 } PickupBorough <null> DropoffBorough <null> pickup_location_id 249 dropoff_location_id 4 5 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 42, dropoff_location_id = 141 } PickupBorough <null> DropoffBorough <null> pickup_location_id 42 dropoff_location_id 141 6 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 163, dropoff_location_id = 87 } PickupBorough <null> DropoffBorough <null> pickup_location_id 163 dropoff_location_id 87 7 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 132, dropoff_location_id = 62 } PickupBorough <null> DropoffBorough <null> pickup_location_id 132 dropoff_location_id 62 8 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 230, dropoff_location_id = 43 } PickupBorough <null> DropoffBorough <null> pickup_location_id 230 dropoff_location_id 43 9 TaxiTrip { PickupBorough = , DropoffBorough = , pickup_location_id = 142, dropoff_location_id = 237 } PickupBorough <null> DropoffBorough <null> pickup_location_id 142 dropoff_location_id 237

In [ ]:



var sw = System.Diagnostics.Stopwatch.StartNew();
var zones = await LoadParquetDataAsync<Zone>(zoneFile);

$"Time to load file and instantiate: {sw.ToString()}"

Time to load file and instantiate: 00:00:00.0050833

In [ ]:
zones.Take(10)

index value 0 Zone { LocationID = 1, Borough = EWR } LocationID 1 Borough EWR 1 Zone { LocationID = 2, Borough = Queens } LocationID 2 Borough Queens 2 Zone { LocationID = 3, Borough = Bronx } LocationID 3 Borough Bronx 3 Zone { LocationID = 4, Borough = Manhattan } LocationID 4 Borough Manhattan 4 Zone { LocationID = 5, Borough = Staten Island } LocationID 5 Borough Staten Island 5 Zone { LocationID = 6, Borough = Staten Island } LocationID 6 Borough Staten Island 6 Zone { LocationID = 7, Borough = Queens } LocationID 7 Borough Queens 7 Zone { LocationID = 8, Borough = Queens } LocationID 8 Borough Queens 8 Zone { LocationID = 9, Borough = Queens } LocationID 9 Borough Queens 9 Zone { LocationID = 10, Borough = Queens } LocationID 10 Borough Queens

First, unoptimized, idea for a query (nothing indexed, taking data as coming from data source)

In [ ]:
var sw = System.Diagnostics.Stopwatch.StartNew();
var manhattanZones = zones.Where(x => x.Borough == "Manhattan").Select(zone => zone.LocationID).ToHashSet();
var query = trips.Where(trip => manhattanZones.Contains(trip.pickup_location_id) || manhattanZones.Contains(trip.dropoff_location_id))
.GroupBy(trip => (trip.pickup_location_id, trip.dropoff_location_id))
.Select(group => new { group.Key.pickup_location_id, group.Key.dropoff_location_id, Count = group.Count() })
.OrderByDescending(x => x.Count)
.Take(5)
.ToList();

Console.WriteLine($" Time to query: {sw.ToString()}");
query

 Time to query: 00:00:20.1773442


index value 0 { pickup_location_id = 237, dropoff_location_id = 236, Count = 536621 } pickup_location_id 237 dropoff_location_id 236 Count 536621 1 { pickup_location_id = 236, dropoff_location_id = 237, Count = 455954 } pickup_location_id 236 dropoff_location_id 237 Count 455954 2 { pickup_location_id = 236, dropoff_location_id = 236, Count = 451805 } pickup_location_id 236 dropoff_location_id 236 Count 451805 3 { pickup_location_id = 237, dropoff_location_id = 237, Count = 435054 } pickup_location_id 237 dropoff_location_id 237 Count 435054 4 { pickup_location_id = 239, dropoff_location_id = 238, Count = 236737 } pickup_location_id 239 dropoff_location_id 238 Count 236737

Idea about Report: Given the borrow, we have the most frequent trips. Normally, we would already read data like this, but in order to avoid reading twice, we just do it here as a followup step.

In [ ]:
var sw = System.Diagnostics.Stopwatch.StartNew();
var numberOfTripsByPickupAndDropOff = trips
                .GroupBy(trip => (trip.pickup_location_id, trip.dropoff_location_id))
                .Select(group => new { group.Key.pickup_location_id, group.Key.dropoff_location_id, Count = group.Count() })
                .OrderByDescending(x => x.Count)
                .ToArray();
Console.WriteLine($" Time to build index: {sw.ToString()}");

 Time to build index: 00:00:23.9330322


In [ ]:
var sw = System.Diagnostics.Stopwatch.StartNew();

var manhattanZones = zones.Where(x => x.Borough == "Manhattan").Select(zone => zone.LocationID).ToHashSet();
var query = numberOfTripsByPickupAndDropOff.Where(trip => manhattanZones.Contains(trip.pickup_location_id) || manhattanZones.Contains(trip.dropoff_location_id))
.Take(5)
.ToList();

Console.WriteLine($" Time to query: {sw.ToString()}");
query

 Time to query: 00:00:00.0006824


index value 0 { pickup_location_id = 237, dropoff_location_id = 236, Count = 536621 } pickup_location_id 237 dropoff_location_id 236 Count 536621 1 { pickup_location_id = 236, dropoff_location_id = 237, Count = 455954 } pickup_location_id 236 dropoff_location_id 237 Count 455954 2 { pickup_location_id = 236, dropoff_location_id = 236, Count = 451805 } pickup_location_id 236 dropoff_location_id 236 Count 451805 3 { pickup_location_id = 237, dropoff_location_id = 237, Count = 435054 } pickup_location_id 237 dropoff_location_id 237 Count 435054 4 { pickup_location_id = 239, dropoff_location_id = 238, Count = 236737 } pickup_location_id 239 dropoff_location_id 238 Count 236737